# Notebook Purpose

This notebook serves as documented walkthrough for the creation of the `illinois-gambling` postgresql database.

### Data Sources

This database contains data from the Illinois Gaming Board's monthly reports for both [Video Gambling](https://www.igb.illinois.gov/VideoReports.aspx) and [Casino Gambling](https://www.igb.illinois.gov/CasinoReports.aspx) as well as demographic data for each Illinois municipality from the 5 year American Community Survey.

<center><img src="../static/schema.png" width="1000"></center>

In [3]:
import sys
sys.path.append('..')
import pandas as pd
import os
import numpy as np
from geopy import geocoders  
import matplotlib.pyplot as plt
from io import BytesIO
from geopy.geocoders import Nominatim
import psycopg2
from requests import get
import zipfile
import censusdata
import json
from geopy.distance import geodesic
from sqlalchemy import create_engine
geolocator = Nominatim(user_agent='Illinois Gambling')

# Create Database

In [4]:
# # Connect to postgresql
# conn = psycopg2.connect(dbname="postgres")
# cursor = conn.cursor()
# # Set configurations 
# conn.autocommit = True 
# conn.set_isolation_level(0)
# # Create empty database
# cursor.execute('CREATE DATABASE illinois_gambling;')
# conn.close()
# Open up new connection to database
conn = psycopg2.connect(dbname="illinois_gambling")
# Create sqlalchemy engine to streamline the sql process
engine = create_engine('postgresql+psycopg2://joel:@localhost:5432/illinois_gambling')

# Casino Table

In [108]:
# Create import path for the casino gambling data
casino_path = os.path.join('data', 'casino_gambling', 'casino_data.csv')
# Read in csv file
casino_table = pd.read_csv(casino_path)
# Change date column from string to datetime
casino_table['date'] = pd.to_datetime(casino_table.date)

# Data Cleaning: 
# The naming of casinos is somewhat inconsistent.
# Below, we map names that deviate from the majority
# so they are consistent across observations
CASINO_REFORMAT = {'Alton': 'ALTON - ARGOSY',
                'East Peoria': 'E. PEORIA - PAR-A-DICE', 
                'Rock Island': 'ROCK ISLAND - JUMERS', 
                 'Joliet - Empress': 'JOLIET - Argosy Empress Casino', 
                 'Metropolis': 'METROPOLIS - HARRAHS',
                "Joliet - Harrah's": 'JOLIET - HARRAHS', 
                'Aurora': 'AURORA - HOLLYWOOD', 
                'E  St  Louis': 'E. PEORIA - PAR-A-DICE', 
                'Elgin': 'ELGIN - GRAND VICTORIA',
                'Joliet - Hollywood': 'JOLIET - HOLLYWOOD', 
                'Des Plaines': 'DES PLAINES - RIVERS CASINO', 
                'METROPOLIS - HARRAHS*': 'METROPOLIS - HARRAHS'}

def casino_map(casino):
    if casino in CASINO_REFORMAT:
        return CASINO_REFORMAT[casino]
    else:
        return casino

casino_table['casino'] = casino_table.casino.apply(casino_map)

# Data Cleaning:
# To make the columns more informative, we seperate the 
# name of the casino from the name of the municipality.
casino_table['municipality'] = casino_table.casino.apply(lambda x: x.split(' - ')[0])
casino_table['casino'] = casino_table.casino.apply(lambda x: x.split(' - ')[1])
casino_table['municipality'] = casino_table.municipality.apply(lambda x: x.title().strip().replace('E.', 'East'))


# The location of the casino is useful information
# for measuring the relationship between video gambling
# and casino performance. The addresses were searched manually 
# Using a google search. While it would be valuable to have a 
# dynamic way of fetching this information, it is unlikely that
# these data points will change. 
casino_addresses = {'Alton': '1 Piasa St, Alton, IL 62002',
'East Peoria':'21 Blackjack Blvd, East Peoria, IL 61611',
'Rock Island':'777 Jumer Dr, Rock Island, IL 61201',
'Joliet': '777 Hollywood Blvd, Joliet, IL 60436',
'Aurora':'100 E Front St, Metropolis, IL 62960',
'East St. Louis': '200 S Front St, East St Louis, IL 62201',
'Elgin': '250 S Grove Ave, Elgin, IL 60120',
'Des Plaines':'3000 S River Rd, Des Plaines, IL 60018',
'Metropolis': '100 E Front St, Metropolis, IL 62960'}

casino_table['address'] = casino_table.municipality.map(casino_addresses)

# The two columns below are aggregates of other columns. Because these numbers are 
# easily calculable from the existing data, they are dropped. 
casino_table.drop(['agr_per_square_foot', 'agr_per_admission'], axis = 1, inplace=True)

def coordinates_from_address(address):
    
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude

coordinates_from_address(casino_table.address[0])

casino_coordinates = {}
for address in casino_table.address.unique():
    try:
        latitude, longitude = coordinates_from_address(address)
        casino_coordinates[address] = (latitude, longitude)
    except:
        continue

casino_coordinates['777 Hollywood Blvd, Joliet, IL 60436'] = (41.479289, -88.145328)
casino_table['latitude'] = casino_table.address.apply(lambda x: casino_coordinates[x][0])
casino_table['longitude'] = casino_table.address.apply(lambda x: casino_coordinates[x][1])


casino_table.rename({'casino': 'name', 'agr': 'adjusted_gross_receipt',
                    'casino_square_feet': 'square_feet'}, axis = 1, inplace=True)

casino_table = casino_table[['address', 'name', 'date', 'municipality',
              'adjusted_gross_receipt', 'square_feet', 'admissions',
               'state_share', 'local_share','latitude', 'longitude']]

casino_table.to_sql('casino', engine, if_exists='append', index=True)

# Video Gambling Table

In [99]:
video_gambling_path = os.path.join('data', 'video_gambling','vg_gambling_09-01-12_10-01-20.csv')
video_gambling_table = pd.read_csv(video_gambling_path)
video_gambling_table['date'] = pd.to_datetime(video_gambling_table.date)
video_gambling_table = video_gambling_table.dropna()

video_gambling_table = video_gambling_table[['date','Establishment', 'License Number', 'Municipality', 'VGT Count',
       'Amount Played', 'Amount Won', 'Net Wager', 'Funds In', 'Funds Out',
       'Net Terminal Income', 'NTI Tax', 'State Share', 'Municipality Share'
       ]]

column_edit = lambda x: x.lower().strip().replace(' ', '_').replace('-', '_')
video_gambling_table.columns = [column_edit(column) for column in video_gambling_table.columns]

video_gambling_table.rename({'vgt_count': 'terminal_count'}, axis = 1, inplace=True)

video_gambling_table.to_sql('video_gambling', engine, if_exists='append', index=True)

# Municipality Table

In [43]:
# Request census geo id dataset

response = get('https://www2.census.gov/programs-surveys/acs/summary_file/2018/data/5_year_by_state/Illinois_All_Geographies_Not_Tracts_Block_Groups.zip')
translated = BytesIO(response.content)
zip_file = zipfile.ZipFile(translated)
csv_raw = zip_file.open('g20185il.csv')
municipality_ids = pd.read_csv(csv_raw, header=None)
municipality_ids = municipality_ids[municipality_ids.iloc[:,2] == 160]

# Collect ids for municipalities
census_ids = {}

counties = video_gambling_table[video_gambling_table.municipality.str.contains('county', case=False)].index
municipalities = video_gambling_table.loc[[x for x in video_gambling_table.index if x not in counties]]
towns_unique = list(municipalities.municipality.unique())
towns_unique += [town for town in casino_table.municipality.unique() if town not in towns_unique]

for town in towns_unique:
    try:
        search = municipality_ids[municipality_ids.iloc[:,49].str.contains(town, case=False)]
        search  = search[~search.iloc[:,49].str.contains('CDP')]
        id_ = search.iloc[:,48].values[0]
        census_ids[town] = id_
    except:
        census_ids[town] = None

error_cities = {'La Salle': 'lasalle',
 'Lamoille': 'la moille',
 'Windsor (Mercer)': 'windsor village' ,
 'Windsor (Shelby)': 'windsor city',
 'Leroy': 'le roy',
 'Depue': 'de pue',
 'Wilmington (Will)': 'wilmington city',
 'Sainte Marie': 'ste. marie',
 'Gulfport': 'Gulf port',
 'Saint Elmo': 'St. Elmo',
 'Whiteash': 'Whiteash',
 'Wilmington (Greene)': 'wilmington village',
 'Garden Prairie': 'Garden Prairie'}

for city in error_cities:
    id_ = municipality_ids[municipality_ids.iloc[:,49].str.contains(error_cities[city], case=False)].iloc[:,48].values[0]
    census_ids[city] = id_

municipal_table = pd.DataFrame()
municipal_table['name'] = towns_unique
municipal_table['id'] = municipal_table.name.map(census_ids)


def find_geo_data(municipality):

    location = geolocator.geocode(f'{municipality}, Illinois')
    if location:
        return location.raw

town_geo = {}

for town in towns_unique:
    town_geo[town] = find_geo_data(town)


def collect_coordinates(town):
    return town_geo[town]['lat'], town_geo[town]['lon'], town_geo[town]['boundingbox']

municipal_table['latitude'], municipal_table['longitude'], municipal_table['boundingbox'] = zip(*municipal_table.name.apply(collect_coordinates))

closest_casinos = []
for idx, row in municipal_table.iterrows():
    town_coor = (row.latitude, row.longitude)
    distances = {}
    for casino in casino_coordinates:
        casino_coor = casino_coordinates[casino]
        miles = geodesic(town_coor, casino_coor).miles
        distances[casino] = miles
    sort = sorted(distances.items(), key=lambda x: x[1])[0][0]
    closest_casinos.append(sort)    



municipal_table['nearest_casino'] = closest_casinos
municipal_table = municipal_table[['name', 'id', 'nearest_casino','latitude', 'longitude', 'boundingbox']]
municipal_table.rename({'id': 'geo_id'}, axis=1, inplace=True)

municipal_table.to_sql('municipality', engine, if_exists='replace', index=False)

In [9]:
munic = pd.read_sql('''select * from municipality;''', engine)

In [12]:
census_ids = munic[['name', 'geo_id']].set_index('name').to_dict()['geo_id']

# Census Data

In [5]:
variables_path = os.path.join('data', 'census_variables.json')
with open(variables_path, 'r') as file:
    variables = json.load(file)

### Add variable lookup table

In [ ]:
lookup

In [6]:
lookup = pd.DataFrame(variables, index=[0]).T.reset_index()
lookup.columns = ['variable', 'description']
lookup['variable'] = lookup.variable.apply(lambda x: x.lower())

lookup.to_sql('demographics_lookup', engine)

### Create Demographics Table

In [15]:
variable_names = list(variables)
variable_names.reverse()
count = 0
errors = {}
for year in range(2012, 2019):
    for town in census_ids:
        try:
            id_ = census_ids[town][-5:]
            geo = censusdata.censusgeo([('state', '17'), ('place', id_)])
            data = censusdata.download('acs5', year, geo, variable_names, key='ef2b118b032f366e377ba482a7e9cdbc8cbfd617')
            data['geo_id'] = census_ids[town]
            data['municipality'] = town
            data['year'] = year
            data = data[['geo_id','municipality', 'year'] + variable_names]
            data.index = [count]
            data.to_sql('demographics', engine, if_exists='append')
            count +=1 
        except:
            if town in errors:
                errors[town] += 1
            else:
                errors[town] = 0
            continue

Unexpected response (URL: https://api.census.gov/data/2012/acs/acs5?get=NAME,B01003_001E,B19326_001E,B15003_001E,B01002_001E,C02003_008E,C02003_007E,C02003_006E,C02003_005E,C02003_004E,C02003_003E,C24050_071E,C24050_057E,C24050_043E,C24050_029E,C24050_015E,C24050_014E,C24050_013E,C24050_012E,C24050_011E,C24050_010E,C24050_009E,C24050_008E,C24050_007E,C24050_006E,C24050_005E,C24050_004E,C24050_003E,C24050_002E&for=place:67158&in=state:17&key=ef2b118b032f366e377ba482a7e9cdbc8cbfd617):  
Unexpected response (URL: https://api.census.gov/data/2013/acs/acs5?get=NAME,B01003_001E,B19326_001E,B15003_001E,B01002_001E,C02003_008E,C02003_007E,C02003_006E,C02003_005E,C02003_004E,C02003_003E,C24050_071E,C24050_057E,C24050_043E,C24050_029E,C24050_015E,C24050_014E,C24050_013E,C24050_012E,C24050_011E,C24050_010E,C24050_009E,C24050_008E,C24050_007E,C24050_006E,C24050_005E,C24050_004E,C24050_003E,C24050_002E&for=place:67158&in=state:17&key=ef2b118b032f366e377ba482a7e9cdbc8cbfd617):  
Unexpected response 

In [147]:
for town in census_ids:
    if census_ids[town][-5:] == '67158':
        print(town)

St. Rose
